In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install gdown

In [2]:
import torch
import gdown
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextStreamer
from trl import SFTTrainer
from sklearn.model_selection import train_test_split

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### **Load Dataset**

In [ ]:
file_path = "data/poem68_dataset.csv"
df = pd.read_csv(file_path)
df

,Unnamed: 0,title,content,source,url
0,0,Việt Bắc,Tiếng ai tha thiết bên cồn\nBâng khuâng trong ...,10-1954\n\nChiến dịch Điện Biên Phủ kết thúc t...,https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu...
1,1,Khi con tu hú,"Khi con tu hú gọi bầy\nLúa chiêm đang chín, tr...","Huế, tháng 7-1939\n\nBài thơ Khi con tu hú đượ...",https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu...
2,2,Tiếng ru,"Con ong làm mật, yêu hoa\nCon cá bơi, yêu nước...",Ba khổ thơ đầu bài này được sử dụng trong SGK ...,https://www.thivien.net/T%E1%BB%91-H%E1%BB%AFu...
3,3,Lịch sử nước ta,"Dân ta phải biết sử ta,\nCho tường gốc tích nư...",Đầu năm 1942\n\nĐể giáo dục tinh thần yêu nước...,https://www.thivien.net/H%E1%BB%93-Ch%C3%AD-Mi...
4,4,Lỡ bước sang ngang,"“- Em ơi, em ở lại nhà,\nVườn dâu em đốn, mẹ g...",Bài thơ này được đăng lần đầu trên Tiểu thuyết...,https://www.thivien.net/Nguy%E1%BB%85n-B%C3%AD...
...,...,...,...,...,...
95,95,Lạ,"Người nào xa lạ đâu đây\nNgười mi thanh sáng, ...",NaN,https://www.thivien.net/Xu%C3%A2n-Di%E1%BB%87u...
96,96,Hồi 09: Kiều rơi vào tay Tú Bà,Rèm trong đã thấy một người bước ra.\nĂn cao l...,Đoạn này ứng với Hồi 8 trong nguyên truyện: “V...,https://www.thivien.net/Nguy%E1%BB%85n-Du/H%E1...
97,97,Áo đỏ em đi trong chiều tuyết trắng,Tất cả đã trở về màu xám\nChỉ còn em và tuyết ...,12.1987\n\nBài thơ đề tặng chị Phan Bích Thiện...,https://www.thivien.net/Ph%E1%BA%A1m-Ti%E1%BA%...
98,98,Hồi 01,"1. Trăm năm một sợi chỉ hồng,\nBuộc người tài ...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Huy-T%E...


In [ ]:
print(df['content'][17])

Bàn tay mẹ chắn mưa sa,
Bàn tay mẹ chặn bão qua mùa màng.
Vẫn bàn tay mẹ dịu dàng,
À ơi này cái trăng vàng ngủ ngon.
À ơi này cái trăng tròn,
À ơi này cái trăng còn nằm nôi…
Bàn tay mẹ thức một đời,
À ơi này cái Mặt Trời bé con…
Mai sau bể cạn non mòn,
À ơi tay mẹ vẫn còn hát ru.
Ru cho mềm ngọn gió thu,
Ru cho tan đám sương mù lá cây.
Ru cho cái khuyết tròn đầy,
Cái thương cái nhớ nặng ngày xa nhau.
Bàn tay mang phép nhiệm màu,
Chắt chiu từ những dãi dầu đấy thôi.
Ru cho sóng lặng bãi bồi,
Mưa không chỗ dột ngoại ngồi vá khâu.
Ru cho đời nín cái đau,
À ơi… mẹ chẳng một câu ru mình.


In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.1, random_state=42)

dataset = {
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(valid_df),
}

print(f"Training size: {len(dataset['train'])}, Validation size: {len(dataset['validation'])}")

Training size: 90, Validation size: 10


### **Model**

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True # 4-bit quantization for memory efficiency

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

### **Preprocess Data**

In [ ]:
import textwrap

def format_poem(example):
    theme = example["title"]
    content = example["content"].strip()

    text = f"""<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Bạn là một AI có khả năng sáng tác thơ lục bát, và phải tuân thủ nghiêm ngặt quy tắc của thơ lục bát:
- Bắt đầu với dòng thơ 6 chữ, tiếp theo là dòng 8 chữ và lặp lại.
- Dòng thơ cuối cùng phải kết thúc bằng dòng 8 chữ.

<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Hãy sáng tác một bài thơ lục bát về chủ đề '{theme}'.

<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
{content}
<|eot_id|>"""

    return {"text": text}

dataset["train"] = dataset["train"].map(format_poem, remove_columns=dataset["train"].column_names)
dataset["validation"] = dataset["validation"].map(format_poem, remove_columns=dataset["validation"].column_names)

print(dataset["train"][0]["text"])

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Bạn là một AI có khả năng sáng tác thơ lục bát, và phải tuân thủ nghiêm ngặt quy tắc của thơ lục bát:
- Bắt đầu với dòng thơ 6 chữ, tiếp theo là dòng 8 chữ và lặp lại.
- Dòng thơ cuối cùng phải kết thúc bằng dòng 8 chữ.

<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Hãy sáng tác một bài thơ lục bát về chủ đề 'Quê tôi'.

<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Quê tôi có gió bốn mùa
Có giăng giữa tháng, có chùa quanh năm.
Chuông hôm, gió sớm, giăng rằm:
Chỉ thanh đạm thế, âm thầm thế thôi.
Tôi về đây, đã lâu rồi,
Nằm trong cô tịch nhớ người phồn hoa
Tóc tơ, mình liễu da ngà,
Một người càng nhớ, càng xa một người
Ngày trông mây trắng bay hoài,
Đêm mơ áo trắng bay dài năm canh
Lòng vàng lạc cánh chim xanh,
Lạc từ cái ý chung tình lạc đi.
Chẳng điên chẳng dại là gì.
Bổng dưng mà biệt mà ly mọi người.
Chưa xa đã nhớ nhau rồi.
Nữa là hơn một tháng giời xa nhau.
Người đi nghỉ mát những đâu,
Đồ Sơn, Tam Đảo, 

In [ ]:
print(dataset["train"][5]["text"])

<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Bạn là một AI có khả năng sáng tác thơ lục bát, và phải tuân thủ nghiêm ngặt quy tắc của thơ lục bát:
- Bắt đầu với dòng thơ 6 chữ, tiếp theo là dòng 8 chữ và lặp lại.
- Dòng thơ cuối cùng phải kết thúc bằng dòng 8 chữ.

<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Hãy sáng tác một bài thơ lục bát về chủ đề 'Lỡ bước sang ngang'.

<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
“- Em ơi, em ở lại nhà,
Vườn dâu em đốn, mẹ già em thương.
Mẹ già một nắng hai sương,
Chị đi một bước trăm đường xót xa.
Cậy em, em ở lại nhà,
Vườn dâu em đốn, mẹ già em thương.
Hôm nay xác pháo đầy đường,
Ngày mai khói pháo còn vương khắp làng.
Chuyến này chị bước sang ngang
Là tan vỡ giấc mộng vàng từ nay.
Rượu hồng em uống cho say,
Vui cùng chị một vài giây cuối cùng.
(Rồi đây sóng gió ngang sông,
Đầy thuyền hận, chị lo không tới bờ)
Miếu thiêng vụng kén người thờ,
Nhà hương khói lạnh, chị nhờ cậy em.
Đêm qua là trắng ba đêm,
Chị th

In [ ]:
def tokenize_function(example):
    tokenized = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_datasets = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
}

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    return_tensors="pt"
)

In [ ]:
# Test batch after collate
batch = [tokenized_datasets["train"][i] for i in range(2)]
collated_batch = data_collator(batch)
print(collated_batch.keys())
print("Input IDs shape:", collated_batch["input_ids"].shape)
print("Labels shape:", collated_batch["labels"].shape)

dict_keys(['input_ids', 'attention_mask', 'labels'])
Input IDs shape: torch.Size([2, 2048])
Labels shape: torch.Size([2, 2048])


### **Training**

In [ ]:
from transformers import TrainingArguments, EarlyStoppingCallback, TrainerCallback

class PrintLogCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(logs)
        return control

training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=70,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=5,
    save_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    seed=3407,
    report_to=[],
    logging_steps=1,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    max_seq_length=max_seq_length,
    data_collator=data_collator,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

Converting eval dataset to ChatML (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

Truncating eval dataset (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5))
trainer.add_callback(PrintLogCallback())

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 90 | Num Epochs = 7
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 70
 "-____-"     Number of trainable parameters = 12,156,928


Step,Training Loss,Validation Loss
5,7.288000,10.098697
10,6.495300,8.057589
15,5.600400,6.887071
20,5.774600,5.946225
25,5.325800,5.591251
30,5.510200,5.625780
35,4.995000,5.566970
40,5.408200,5.533537
45,5.200600,5.537234
50,5.200200,5.533082


{'loss': 10.2236, 'grad_norm': 17.03887939453125, 'learning_rate': 4e-05, 'epoch': 0.08888888888888889}
{'loss': 9.5634, 'grad_norm': 14.731060981750488, 'learning_rate': 8e-05, 'epoch': 0.17777777777777778}
{'loss': 8.9857, 'grad_norm': 14.595236778259277, 'learning_rate': 0.00012, 'epoch': 0.26666666666666666}
{'loss': 9.8523, 'grad_norm': 20.9027099609375, 'learning_rate': 0.00016, 'epoch': 0.35555555555555557}


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


{'loss': 7.288, 'grad_norm': 8.059837341308594, 'learning_rate': 0.0002, 'epoch': 0.4444444444444444}
{'eval_loss': 10.0986967086792, 'eval_runtime': 18.3462, 'eval_samples_per_second': 0.545, 'eval_steps_per_second': 0.273, 'epoch': 0.4444444444444444}
{'loss': 7.4039, 'grad_norm': 10.838139533996582, 'learning_rate': 0.00019692307692307696, 'epoch': 0.5333333333333333}
{'loss': 7.0171, 'grad_norm': 6.022526741027832, 'learning_rate': 0.00019384615384615385, 'epoch': 0.6222222222222222}
{'loss': 7.262, 'grad_norm': 4.605217456817627, 'learning_rate': 0.0001907692307692308, 'epoch': 0.7111111111111111}
{'loss': 6.5078, 'grad_norm': 2.537604331970215, 'learning_rate': 0.0001876923076923077, 'epoch': 0.8}
{'loss': 6.4953, 'grad_norm': 2.494957208633423, 'learning_rate': 0.00018461538461538463, 'epoch': 0.8888888888888888}
{'eval_loss': 8.057588577270508, 'eval_runtime': 16.5467, 'eval_samples_per_second': 0.604, 'eval_steps_per_second': 0.302, 'epoch': 0.8888888888888888}
{'loss': 6.3695

TrainOutput(global_step=70, training_loss=5.608320222582136, metrics={'train_runtime': 1662.7157, 'train_samples_per_second': 0.337, 'train_steps_per_second': 0.042, 'total_flos': 1.84366607106048e+16, 'train_loss': 5.608320222582136})

In [ ]:
save_path = "poemlucbat_gen_finetuned_llama3.2"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=save_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

## **Inference**

In [ ]:
def generate_poem(prompt, max_new_tokens=200):
    system_prompt = textwrap.dedent("""\
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        Bạn là một AI có khả năng sáng tác thơ lục bát, và phải tuân thủ nghiêm ngặt quy tắc của thơ lục bát:
            - Bắt đầu với dòng thơ 6 chữ, tiếp theo là dòng thơ 8 chữ và lặp lại.
            - Dòng thơ cuối cùng phải kết thúc bằng dòng thơ 8 chữ.
        <|eot_id|>
        """)

    user_prompt = textwrap.dedent(f"""\
        <|start_header_id|>user<|end_header_id|>
        Hãy sáng tác một bài thơ lục bát về chủ đề '{prompt}'.
        <|eot_id|>

        <|start_header_id|>assistant<|end_header_id|>
        """)

    input_text = system_prompt + "\n" + user_prompt

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_seq_length
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}

    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.8,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.1
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_poem = generated_text.replace(input_text, "").strip()
    return generated_poem

In [ ]:
theme = "Mừng ngày quốc tế phụ nữ"
print(f"Chủ đề: {theme}\n")
print(generate_poem(theme))

Chủ đề: Mừng ngày quốc tế phụ nữ

system
Bạn là một AI có khả năng sáng tác thơ lục bát, và phải tuân thủ nghiêm ngặt quy tắc của thơ lục bát:
    - Bắt đầu với dòng thơ 6 chữ, tiếp theo là dòng thơ 8 chữ và lặp lại.
    - Dòng thơ cuối cùng phải kết thúc bằng dòng thơ 8 chữ.


user
Hãy sáng tác một bài thơ lục bát về chủ đề 'Mừng ngày quốc tế phụ nữ'.


assistant
Nàng sao trời cao tăm,
Một mình sinh hoạt đanh nghịch trong vườn.
Nhà cửa rộng lớn,
Mà nàng ai ở đây?
Cũng chẳng được lòng nhớ mình,
Một người đã xa cách.
Mái nhà để chén nước sương,
Lạc vào mây dông giang hồ.
Mùa xuân hoa nở rộ,
Lá cây xanh tươi màu.
Chưa đến xuân thôi,
Mùa thu mà mùa thu.
Thềm dưới bờ sông thẳm,
Đất đỏ mỏng đất vàng.
Có lẽ năm tháng mười mấy,
Mãi mới làm một lần.
Bên đường lặng lẽ,
Sao trời biết đâu?
Vậy còn đâu những lời nói,
Đã ra từ người mẹ.
Vì sao anh ta lại đi,
Để lại con gái.
Sâu trong đêm tối,
Tìm tôi thì sao?
Tôi sẽ đến đây,
Đi tìm em ấy!
Kh


In [ ]:
!pip install gradio

import gradio as gr

def gradio_generate_poem(theme):
    return generate_poem(theme)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [ ]:
demo = gr.Interface(
    fn=gradio_generate_poem,
    inputs=gr.Textbox(label="Nhập chủ đề thơ"),
    outputs=gr.Textbox(label="Bài thơ lục bát"),
    title="AI Sáng Tác Thơ Lục Bát",
    description="Nhập chủ đề để AI sáng tác thơ lục bát",
    theme="compact",
)

demo.launch(share=True)